# Libraries

In [ ]:
# google drive access
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Colab Notebooks

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [ ]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import torchvision
from sklearn.metrics import accuracy_score

# install and import the torchinfo library
!pip install torchinfo
from torchinfo import summary

# Model

In [ ]:
class CCM(nn.Module):
    """
    Channel wise calibration model:
        y = x * Sigmoid(BN(Group_Conv(GAP(x))))
    """
    def __init__(self, in_channel, out_channel):
        super(CCM, self).__init__()
        self.adapool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=1, padding=0, groups=in_channel, bias=False)
        self.bn = nn.BatchNorm2d(out_channel)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        out = self.adapool(x)
        out = self.conv(out)
        out = self.bn(out)
        out = self.sig(out)
        out = x * out
        return out


class SCM(nn.Module):
    """
    Spatial Calibration Module:
        y = x + Group_Conv(x)
    """
    def __init__(self, channel):
        super(SCM, self).__init__()
        self.conv = nn.Conv2d(channel, channel, kernel_size=3,stride=1, padding=1, groups=channel, bias=False)

    def forward(self, x):
        out = self.conv(x)
        out = x + out
        return out


class Shortcut(nn.Module):
    """
    Modified skip connections in ResNet 
    with calibration modules.
    """
    def __init__(self, in_channel, out_channel, stride=1):
        super(Shortcut, self).__init__()
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride, bias=False)
        self.scm = SCM(out_channel)
        self.bn = nn.BatchNorm2d(out_channel)
        self.ccm = CCM(out_channel, out_channel)

    def forward(self, x):
        out = self.conv(x)
        out = self.scm(out)
        out = self.bn(out)
        out = self.ccm(out)
        return out


class BasicBlock(nn.Module):
    expansion = 1
    """
    Convolutional blocks of ResNet with 
    skip connections and calibration modules.
    """
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        # SCM and CCM for conv1
        self.scm1 = SCM(planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.ccm1 = CCM(planes, planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        # SCM and CCM for conv2
        self.scm2 = SCM(planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.ccm2 = CCM(planes, planes)

        self.shortcut = None
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = Shortcut(in_planes, self.expansion*planes, stride)

    def forward(self, x):
        # conv1 forward pass
        out = self.conv1(x)
        out = self.scm1(out)
        out = self.bn1(out)
        out = self.ccm1(out)
        out = F.relu(out)
        # conv2 forward pass
        out = self.conv2(out)
        out = self.scm2(out)
        out = self.bn2(out)
        out = self.ccm2(out)
        if self.shortcut is not None:
            out += self.shortcut(x)
        else:
            out += x
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        # input convolutions
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.scm1 = SCM(64)
        self.bn1 = nn.BatchNorm2d(64)
        self.ccm1 = CCM(64, 64)
        # residual blocks
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        # classifier head
        self.classifier = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        # forward pass of input conv
        out = self.conv1(x)
        out = self.scm1(out)
        out = self.bn1(out)
        out = self.ccm1(out)
        out = F.relu(out)
        # forward pass of residual blocks
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        # forward pass of classifier head
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out


def get_resnet18(num_classes=10):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

# Utils

In [ ]:
def freeze_noncalibration_layers(model):
    for child in model.children():
        # Base model convs are frozen
        # print('Child: ', type(child))
        if type(child) == nn.Conv2d:
            froze_this_layer(child, tab='')
        # Search for basic blocks in Sequentials
        elif type(child) == nn.Sequential:
            for child2 in child.children():
                # print('\t Child2: ', type(child2))
                if type(child2) == nn.Conv2d:
                    froze_this_layer(child2, tab='\t ')
                # Search for Shortcuts in Basic Blocks
                elif type(child2) == BasicBlock:
                    for child3 in child2.children():
                        # print('\t\t Child3: ', type(child3))
                        # Base model convs are frozen
                        if type(child3) == nn.Conv2d:
                            froze_this_layer(child3, tab='\t\t ')
                        elif type(child3) == Shortcut:
                            # Base model convs are frozen
                            for child4 in child3.children():
                                # print('\t\t\t Child4: ', type(child4))
                                if type(child4) == nn.Conv2d:
                                    froze_this_layer(child4, tab='\t\t\t ')

def froze_this_layer(layer, tab=''):
    for param in layer.parameters():
        # print(tab + '********* Conv2d frozen *********')
        param.requires_grad = False


def accuracy(y, y_hat):
    y_hat = np.argmax(y_hat, axis=1)
    y = np.squeeze(y)
    acc = accuracy_score(y, y_hat)
    return acc

# Dataset

In [ ]:
def get_dataloader(task_no=0, batch_size=128, subset='train', val_ratio=0.1, vis=False):
    if subset == 'test':
        shuffle = False
        raw_data = torchvision.datasets.CIFAR100(root='./data', train=False, download=True)
        # task specific params
        classes = list(raw_data.class_to_idx.keys())
        task_classes = classes[task_no*10:10+task_no*10]
        task_class_labels = [int(raw_data.class_to_idx[class_]) for class_ in task_classes]
    else:
        raw_data = torchvision.datasets.CIFAR100(root='./data', train=True, download=True)
        # task specific params
        classes = list(raw_data.class_to_idx.keys())
        task_classes = classes[task_no*10:10+task_no*10]
        task_class_labels = [int(raw_data.class_to_idx[class_]) for class_ in task_classes]
        # split to train-validation datasets generator seed set for reproducibility
        val_len = int(len(raw_data) * val_ratio)
        train_len = int(len(raw_data) - val_len)
        raw_train, raw_val = random_split(raw_data, [train_len, val_len], generator=torch.Generator().manual_seed(42))
        
        if subset == 'train':
            shuffle = True
            raw_data = raw_train
        elif subset == 'val':
            shuffle = False
            raw_data = raw_val

    # create custom dataset
    transforms = torchvision.transforms.Compose(
        [
         torchvision.transforms.ToTensor(), 
         torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ]
    )
    task_data = CustomCIFAR100(raw_data, task_classes, task_class_labels, task_no=task_no, transforms=transforms)
    print('Num. samples ({}) in task dataset: {}'.format(subset, len(task_data)))

    # set batch_sizes
    if subset == 'train':
        bsize = batch_size
    else:
        bsize = len(task_data)
    # create dataloader
    dataloader = DataLoader(task_data, batch_size=bsize, shuffle=shuffle, num_workers=2)
    if vis:
        num_classes = 10
        samples_per_class = 7
        task_data_copy = CustomCIFAR100(raw_data, task_classes, task_class_labels, task_no=task_no)
        visualize_samples(task_data_copy, task_classes, num_classes, samples_per_class)

    return dataloader
    

class CustomCIFAR100(Dataset):
    def __init__(self, dataset, class_names, class_idxs, task_no=0, transforms=None):
        super(CustomCIFAR100, self).__init__()
        self.dataset = dataset
        self.task_no = task_no
        self.class_names = class_names
        self.class_idxs = class_idxs
        self.transforms = transforms
        self.idxs = [idx for idx in range(len(dataset)) if (self.dataset[idx][1] in self.class_idxs)]
        self.labels = [self.dataset[idx][1] - 10*self.task_no for idx in self.idxs]

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img, _ = self.dataset[self.idxs[idx]]
        y = self.labels[idx]

        # convert labels to torch tensor too
        if not isinstance(y, np.ndarray):
            y = np.array(y)
            y = torch.from_numpy(y)

        # apply input image transformations
        if self.transforms:
            img = self.transforms(img)
        return img, y


def visualize_samples(dataset, classes, num_classes, samples_per_class):
    plt.figure(figsize=(12,8))
    for y, class_name in enumerate(classes):
        idxs = np.flatnonzero(np.array(dataset.labels) == y)
        idxs = np.random.choice(idxs, samples_per_class, replace=False)
        for i, idx in enumerate(idxs):
            plt_idx = i * num_classes + y + 1
            plt.subplot(samples_per_class, num_classes, plt_idx)
            img, _ = dataset[idx]
            plt.imshow(img)
            plt.axis('off')
            if i == 0:
                plt.title(class_name)
    plt.show()

In [ ]:
"""
cifar100_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True)
keys = list(cifar100_dataset.class_to_idx.keys())
for task_no in range(10):
    start_idx = task_no * 10
    end_idx = start_idx + 10
    group_classes = keys[start_idx:end_idx]
    class_idxs = [cifar100_dataset.class_to_idx[key] for key in group_classes]
    print('||*** Task-{} ***|| \n \t classnames:{} \n \t class_idx:{}'.format(task_no, group_classes, class_idxs))

train_loader3 = get_dataloader(task_no=3, batch_size=128, subset='train', val_ratio=0.2, vis=True)
val_loader5 = get_dataloader(task_no=5, subset='val', val_ratio=0.2, vis=True)
test_loader7 = get_dataloader(task_no=7, subset='test', vis=True)
"""

# Main

In [ ]:
if torch.cuda.is_available():
  print("Cuda (GPU support) is available and enabled!")
  device = torch.device("cuda")
else:
  print("Cuda (GPU support) is not available")
  device = torch.device("cpu")

Cuda (GPU support) is available and enabled!


In [ ]:
def train(verbose=True):
    # create ResNet18 model
    model = ResNet(BasicBlock, [2,2,2,2], num_classes=10)
    model = model.to(device)

    # tensorboard logger
    writer = SummaryWriter(log_dir='runs/CIFAR100_experiment')
    model_path = './models/CIFAR100_experiment/'
    # training parameters
    batch_size = 64
    lr = 1e-2
    epochs_per_task = 50

    for task_no in range(10):
        task_tag = 'task_' + str(task_no)
        # freeze layers other than CCM and SCM,
        # add new classifier head for next task
        if task_no > 0:
            print('************* Re-calibrating model parameters *************')
            model = model.to('cpu')
            freeze_noncalibration_layers(model)
            model.classifier = nn.Linear(512*BasicBlock.expansion, 10)
            model = model.to(device)

        # check layer freezing
        num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print('************* Task: {}, Number of Trainable Parameters: {} *************'.format(task_no, num_trainable_params))

        # get dataset for task no
        task_train_loader = get_dataloader(task_no=task_no, subset='train', batch_size=batch_size)
        task_val_loader = get_dataloader(task_no=task_no, subset='val')

        # loss, optimizer and lr-scheduler are identical for each task
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr, momentum=0.9)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 30], gamma=0.1)

        # training iterations
        tr_loss_hist, val_loss_hist, val_acc_hist = [], [], []
        num_iter = 0
        for e in range(epochs_per_task):
            # put model to training mode after each eval
            model.train()
            # training for an epoch
            tqdm_desc = 'Epoch ' + str(e+1) + '/' + str(epochs_per_task)
            for _, data in tqdm(enumerate(task_train_loader), ascii=True, desc=tqdm_desc):
                # get the data
                x, y = data
                x, y = x.to(device), y.to(device)

                # clear gradients
                optimizer.zero_grad()

                # calculate loss
                yhat = model(x)
                loss = criterion(yhat, y)

                # backprop
                loss.backward()

                # update parameters
                optimizer.step()

                # store training loss
                tr_loss_hist.append(loss.item())
                # save training loss per minibatch iterations
                writer.add_scalar('tr_loss/'+task_tag, loss.item(), num_iter)
                num_iter += 1

            # get average training loss for last 5 iterations
            tr_loss = np.sum(tr_loss_hist[:-6:-1])/5
            # update learning-rate
            scheduler.step()
            
            # validation after each epoch
            model.eval()
            with torch.no_grad():
                for i, data in enumerate(task_val_loader):
                    x, y = data
                    x, y = x.to(device), y.to(device)
                    yhat = model(x)
                # compute validation metrics
                val_loss = criterion(yhat, y) .item()
                val_acc = accuracy(y.to('cpu').numpy(), yhat.to('cpu').numpy())

            # save validation scores
            val_loss_hist.append(val_loss)
            val_acc_hist.append(val_acc)
            # save to tensorboard
            writer.add_scalar('val_loss/'+task_tag, val_loss, e)
            writer.add_scalar('val_acc/'+task_tag, val_acc, e)
            # display training info
            if verbose:
                print('tr_loss:{} || val_loss:{} || val_acc:{}'.format(tr_loss, val_loss, val_acc))
                
        # save the model for current task
        print('************* Saving model for Task-{} *************'.format(task_no))
        model_filename = 'cifar100_model_task_' + str(task_no) + '.pt'
        torch.save(model, model_path + model_filename)
        print('************* End of Task-{} training *************'.format(task_no))

In [ ]:
train()

************* Task: 0, Number of Trainable Parameters: 11231562 *************
Files already downloaded and verified
Num. samples (train) in task dataset: 4494
Files already downloaded and verified
Num. samples (val) in task dataset: 506


Epoch 1/50: 71it [00:08,  8.65it/s]


tr_loss:1.9765633821487427 || val_loss:1.93231999874115 || val_acc:0.2826086956521739


Epoch 2/50: 71it [00:07,  8.93it/s]


tr_loss:1.6725311994552612 || val_loss:1.6142418384552002 || val_acc:0.45652173913043476


...

tr_loss:1.472704529762268 || val_loss:1.5472254753112793 || val_acc:0.46234309623430964


Epoch 24/50: 71it [00:05, 12.80it/s]


tr_loss:1.5202971935272216 || val_loss:1.5400484800338745 || val_acc:0.4874476987447699


Epoch 25/50: 25it [00:02, 11.91it/s]


KeyboardInterrupt: ignored

In [ ]:
def test():
    model_path = './models/CIFAR100_experiment/'
    for task_no in range(10):
        # load models per task
        series_num = 'cifar100_model_task_' + str(task_no) + '.pt'
        model = torch.load(model_path + series_num)
        model = model.to(device)
        model.eval()
        # get task specific test data
        test_loader = get_dataloader(subset='test', task_no=task_no)
        criterion = nn.CrossEntropyLoss()

        with torch.no_grad():
            for _, data in enumerate(test_loader):
                x, y = data
                x, y = x.to(device), y.to(device)
                yhat = model(x)

                test_loss = criterion(yhat, y).item()
                test_acc = accuracy(y.to('cpu').numpy(), yhat.to('cpu').numpy())
            print('Task-{} || test_loss:{} || test_acc:{}'.format(task_no, test_loss, test_acc))

In [ ]:
test()

Files already downloaded and verified
Num. samples (test) in task dataset: 1000
Task-0 || 	 test_loss:1.118534803390503 || 	 test_acc:0.725
Files already downloaded and verified
Num. samples (test) in task dataset: 1000
Task-1 || 	 test_loss:1.2035876512527466 || 	 test_acc:0.584
Files already downloaded and verified
Num. samples (test) in task dataset: 1000
Task-2 || 	 test_loss:1.0089589357376099 || 	 test_acc:0.665
Files already downloaded and verified
Num. samples (test) in task dataset: 1000
Task-3 || 	 test_loss:1.098892331123352 || 	 test_acc:0.642
Files already downloaded and verified
Num. samples (test) in task dataset: 1000
Task-4 || 	 test_loss:0.9535859227180481 || 	 test_acc:0.676
Files already downloaded and verified
Num. samples (test) in task dataset: 1000
Task-5 || 	 test_loss:0.8811346292495728 || 	 test_acc:0.704
Files already downloaded and verified
Num. samples (test) in task dataset: 1000
Task-6 || 	 test_loss:0.9358561038970947 || 	 test_acc:0.679
Files already d